Cleaning Train Data and merging the 4 Train Files. 

In [0]:
import pandas as pd
import numpy as np
import collections
import glob

This part of code was ran 4 times for 4 train files.

In [0]:
# Reading the customer list for wich the test data is available
cust = pd.read_csv("cust.csv")

In [6]:
# Read the train file 
train = pd.read_csv("train4.csv")
train = train[['SHOP_WEEK','PROD_CODE','CUST_CODE','BASKET_ID','BASKET_NUM_x']]
train

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x
0,200646,PRD0900385,CUST0000000001,994104000154902,1
1,200646,PRD0900440,CUST0000000001,994104000154902,1
2,200646,PRD0900569,CUST0000000001,994104000154902,1
3,200646,PRD0900830,CUST0000000001,994104000154902,1
4,200646,PRD0901259,CUST0000000001,994104000154902,1
...,...,...,...,...,...
25664181,200703,PRD0904962,CUST0000999999,994104900828925,7
25664182,200703,PRD0903678,CUST0000999999,994104900828925,7
25664183,200704,PRD0900603,CUST0000999999,994105000808993,8
25664184,200704,PRD0901265,CUST0000999999,994105000808993,8


In [9]:
# Merge train data and customer data to keep train data for customers whose test data is available
train = pd.merge(tr, cust, on = "CUST_CODE", how = 'left')
train = train[train['FLAG'] == 1] # If flag is 1 keep the customer else remove
train = train.reset_index(drop = True)
train

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,FLAG
0,200646,PRD0900385,CUST0000000001,994104000154902,1,1.0
1,200646,PRD0900440,CUST0000000001,994104000154902,1,1.0
2,200646,PRD0900569,CUST0000000001,994104000154902,1,1.0
3,200646,PRD0900830,CUST0000000001,994104000154902,1,1.0
4,200646,PRD0901259,CUST0000000001,994104000154902,1,1.0
...,...,...,...,...,...,...
25320029,200703,PRD0904962,CUST0000999999,994104900828925,7,1.0
25320030,200703,PRD0903678,CUST0000999999,994104900828925,7,1.0
25320031,200704,PRD0900603,CUST0000999999,994105000808993,8,1.0
25320032,200704,PRD0901265,CUST0000999999,994105000808993,8,1.0


In [0]:
# Export the cleaned train data for all 4 train files
train.to_csv("cleaned_train4.csv", index=False)

# Run the above code lines for all 4 train files.

After doing the above process 4 times i.e running the code 4 times for each uncleaned train, combine them to form 1 train file

In [3]:
#Reading all cleaned training files

path = "" # path to the folder with the 4 cleaned train files

all_files = glob.glob(path + "/*.csv")

l = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df = df.drop(['FLAG'], axis=1)
    l.append(df)
    print(l)

data = pd.concat(l, axis=0, ignore_index=True)
data = data.dropna()
data.head()

[          SHOP_WEEK   PROD_CODE       CUST_CODE        BASKET_ID  BASKET_NUM_x
0            200608  PRD0900121  CUST0000000001  994100200154444             1
1            200608  PRD0900186  CUST0000000001  994100200154444             1
2            200608  PRD0900398  CUST0000000001  994100200154444             1
3            200608  PRD0900424  CUST0000000001  994100200154444             1
4            200608  PRD0900440  CUST0000000001  994100200154444             1
...             ...         ...             ...              ...           ...
24310848     200616  PRD0903474  CUST0000999999  994101000809421             2
24310849     200616  PRD0903873  CUST0000999999  994101000809421             2
24310850     200616  PRD0904358  CUST0000999999  994101000809421             2
24310851     200617  PRD0900603  CUST0000999999  994101100792416             3
24310852     200617  PRD0903835  CUST0000999999  994101100792416             3

[24310853 rows x 5 columns]]
[          SHOP_WEEK 

,SHOP_WEEK,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x
0,200608,PRD0900121,CUST0000000001,994100200154444,1
1,200608,PRD0900186,CUST0000000001,994100200154444,1
2,200608,PRD0900398,CUST0000000001,994100200154444,1
3,200608,PRD0900424,CUST0000000001,994100200154444,1
4,200608,PRD0900440,CUST0000000001,994100200154444,1


In [0]:
data = data.drop_duplicates()
data = data.sort_values(["CUST_CODE", "SHOP_WEEK", "BASKET_ID"],ascending=True)
data = data.reset_index(drop = True)

In [6]:
# Getting the shop week in which the test order of each user exists.
basket = pd.read_csv("last_basket.csv")
basket = basket[["CUST_CODE","SHOP_WEEK"]]
basket.head()

,CUST_CODE,SHOP_WEEK
0,CUST0000000001,200706
1,CUST0000000009,200705
2,CUST0000000010,200705
3,CUST0000000012,200642
4,CUST0000000020,200705


In [9]:
# Merging the train data and last basket week of each user
data = pd.merge(data, basket, on = 'CUST_CODE', how = 'left')
data = data.reset_index(drop = True)
data.head()

,SHOP_WEEK_x,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,SHOP_WEEK_y
0,200608,PRD0900121,CUST0000000001,994100200154444,1,200706
1,200608,PRD0900186,CUST0000000001,994100200154444,1,200706
2,200608,PRD0900398,CUST0000000001,994100200154444,1,200706
3,200608,PRD0900424,CUST0000000001,994100200154444,1,200706
4,200608,PRD0900440,CUST0000000001,994100200154444,1,200706


In [4]:
# Keeping all the orders that belong to the week that are before the test order week for that specific user.
data = data[data.SHOP_WEEK_x < data.SHOP_WEEK_y]
data.head()

,SHOP_WEEK_x,PROD_CODE,CUST_CODE,BASKET_ID,BASKET_NUM_x,SHOP_WEEK_y
0,200608,PRD0900121,CUST0000000001,994100200154444,1,200706
1,200608,PRD0900186,CUST0000000001,994100200154444,1,200706
2,200608,PRD0900398,CUST0000000001,994100200154444,1,200706
3,200608,PRD0900424,CUST0000000001,994100200154444,1,200706
4,200608,PRD0900440,CUST0000000001,994100200154444,1,200706


In [0]:
data = data[["SHOP_WEEK_x","PROD_CODE","CUST_CODE","BASKET_ID"]]
data = data.sort_values(["CUST_CODE", "SHOP_WEEK_x", "BASKET_ID"],ascending=True)

In [6]:
# Giving new order numbers to the combined train data of all users.
bnum = []
cust = []
bas = []
n = 1
for i in range(len(data)):
    print(len(bnum))
    if data["CUST_CODE"][i] in cust:
        if data["BASKET_ID"][i] in bas:
            bnum.append(n)
        else:
            bas.append(data["BASKET_ID"][i])
            n = n+1
            bnum.append(n)
            
    else:
        cust = []
        bas = []
        cust.append(data["CUST_CODE"][i])
        bas.append(data["BASKET_ID"][i])
        n = 1
        bnum.append(n)
        
data['BASKET_NUM'] = bnum

Streaming output truncated to the last 5000 lines.
93095296
93095297
93095298
93095299
93095300
93095301
93095302
93095303
93095304
93095305
93095306
93095307
93095308
93095309
93095310
93095311
93095312
93095313
93095314
93095315
93095316
93095317
93095318
93095319
93095320
93095321
93095322
93095323
93095324
93095325
93095326
93095327
93095328
93095329
93095330
93095331
93095332
93095333
93095334
93095335
93095336
93095337
93095338
93095339
93095340
93095341
93095342
93095343
93095344
93095345
93095346
93095347
93095348
93095349
93095350
93095351
93095352
93095353
93095354
93095355
93095356
93095357
93095358
93095359
93095360
93095361
93095362
93095363
93095364
93095365
93095366
93095367
93095368
93095369
93095370
93095371
93095372
93095373
93095374
93095375
93095376
93095377
93095378
93095379
93095380
93095381
93095382
93095383
93095384
93095385
93095386
93095387
93095388
93095389
93095390
93095391
93095392
93095393
93095394
93095395
93095396
93095397
93095398
93095399
93095400
9309

In [0]:
# Export Final Train data
data.to_csv("final_train.csv")